In [8]:
import os
import warnings
warnings.filterwarnings('ignore')
from os import path

import pandas as pd
from implicit.nearest_neighbours import CosineRecommender
from rectools.dataset import Dataset
from rectools import Columns
from rectools.models.popular import PopularModel
from rectools.model_selection import TimeRangeSplit



In [11]:
# Train model flag
from pathlib import Path
import numpy as np

DATA_PATH = Path("kion_train")


RETRAIN = False

users = pd.read_csv(DATA_PATH / 'users.csv')
items = pd.read_csv(DATA_PATH / 'items.csv')
interactions = pd.read_csv(DATA_PATH / 'interactions.csv')


Columns.Datetime = 'last_watch_dt'


# примеведем все к Datetime
interactions.drop(interactions[interactions[Columns.Datetime].str.len() != 10].index, inplace=True)
interactions[Columns.Datetime] = pd.to_datetime(interactions[Columns.Datetime], format='%Y-%m-%d')
max_date = interactions[Columns.Datetime].max()

# if user wathed more than 10, 3, 1, percent - set 3 балла этому взаим-ию
# if less - 1 балл
interactions[Columns.Weight] = np.where(interactions['watched_pct'] > 10, 3, 1)


# отправим в test  max  interaction for the last week
train = interactions[interactions[Columns.Datetime] < max_date - pd.Timedelta(days=7)].copy()
test = interactions[interactions[Columns.Datetime] >= max_date - pd.Timedelta(days=7)].copy()


train.drop(train.query("total_dur < 300").index, inplace=True)

# отфильтруем cold пользователей from test
cold_users = set(test[Columns.User]) - set(train[Columns.User])

# in test only users with info about interaction
test.drop(test[test[Columns.User].isin(cold_users)].index, inplace=True)


#users.fillna('Unknown', inplace=True)
#users = users.loc[users[Columns.User].isin(train[Columns.User])].copy()


#user_features_frames = []
#for feature in ["sex", "age", "income"]:
#    feature_frame = users.reindex(columns=[Columns.User, feature])
#    feature_frame.columns = ["id", "value"]
#    feature_frame["feature"] = feature
#    user_features_frames.append(feature_frame)
#user_features = pd.concat(user_features_frames)
#user_features.head(10)

#items = items.loc[items[Columns.Item].isin(train[Columns.Item])].copy()

#items["genre"] = items["genres"].str.lower().str.replace(", ", ",", regex=False).str.split(",")
#genre_feature = items[["item_id", "genre"]].explode("genre")
#genre_feature.columns = ["id", "value"]
#genre_feature["feature"] = "genre"
#genre_feature.head(4)

#content_feature = items.reindex(columns=[Columns.Item, "content_type"])
#content_feature.columns = ["id", "value"]
#content_feature["feature"] = "content_type"

#item_features = pd.concat((genre_feature, content_feature))
#item_features.head(3)



In [19]:
dataset = Dataset.construct(
            interactions_df=interactions,
            user_features_df=None,
            item_features_df=None,
        )

from rectools.metrics import Precision, Recall, MAP, calc_metrics


In [15]:
metrics_name = {
    'Precision': Precision,
    'Recall': Recall,
    'MAP': MAP,
}

metrics = {}
for metric_name, metric in metrics_name.items():
    for k in range(1, 11):
        metrics[f'{metric_name}@{k}'] = metric(k=k)

In [16]:
K_RECOS = 10      # num of recomendations
RAND_ST = 777
THREADS = 16
N_FACTORS = (32, 64, 128, 160)      # num of factors
ITERATIONS = (15, 20)



# Fit Popular model

In [20]:
pop = PopularModel()
pop.fit(dataset)

# Get recomendations

In [22]:
popular_reco_df = pop.recommend(
    dataset.user_id_map.external_ids[:1],
    dataset=dataset,
    k=K_RECOS,
    filter_viewed=False  # True - throw away some items for each user
)

from rectools.models import PopularModel

# Save recomendations df

In [24]:


popular_reco_df.to_csv('/home/iuliiasolomennikova/!!!RecSysService/notebooks/offline_recommend_df/popular', encoding='utf-8', index=False)
